In [141]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [142]:
plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
playerplay = pd.read_csv('player_play.csv')
df = pd.read_csv('tracking_week_1.csv')

In [143]:
playscols = ['gameId','playId', 'possessionTeam','defensiveTeam','quarter', 'down', 'yardsToGo', 'offenseFormation', 'receiverAlignment','passResult','playAction', 'timeToThrow','pff_passCoverage','pff_manZone']

In [144]:
filteredplays = plays.loc[(plays['passResult'].notna())&(plays['pff_manZone']=="Man"), playscols]

In [145]:
playerplayscols = ['gameId', 'playId', 'nflId','teamAbbr','inMotionAtBallSnap', 'motionSinceLineset', 'routeRan', 'hadPassReception', 'receivingYards']

In [146]:
filteredplayerplay = playerplay[playerplayscols][playerplay['routeRan'].notna()]

In [147]:
merged_playerplay = pd.merge(
    filteredplays,
    filteredplayerplay[['gameId','playId','nflId','inMotionAtBallSnap','motionSinceLineset','routeRan','hadPassReception','receivingYards']],
    how='inner',
    left_on=['gameId', 'playId'],
    right_on=['gameId', 'playId']
)

In [148]:
route_runners = playerplay[playerplay['routeRan'].notna()][['gameId', 'playId', 'nflId', 'routeRan']]
defenders = playerplay[playerplay['pff_primaryDefensiveCoverageMatchupNflId'].notna()][['gameId', 'playId', 'nflId', 'pff_primaryDefensiveCoverageMatchupNflId']]
defenders = defenders.rename(columns={'pff_primaryDefensiveCoverageMatchupNflId': 'routeRunnerId'})

In [149]:
merged_df = defenders.merge(route_runners, left_on=['gameId', 'playId', 'routeRunnerId'], right_on=['gameId', 'playId', 'nflId'])

In [150]:
merged_df.drop(columns=['nflId_y'], inplace=True)
merged_df.rename(columns={'nflId_x':'defenderId'}, inplace=True)
merged_df

,gameId,playId,defenderId,routeRunnerId,routeRan
0,2022090800,56,38577,47879.0,IN
1,2022090800,56,53532,47879.0,IN
2,2022090800,56,43294,42489.0,IN
3,2022090800,56,47844,44985.0,OUT
4,2022090800,122,38577,47857.0,FLAT
...,...,...,...,...,...
39916,2022110700,3480,42361,54476.0,GO
39917,2022110700,3480,46077,46160.0,IN
39918,2022110700,3480,52627,46160.0,IN
39919,2022110700,3480,52436,52942.0,OUT


In [151]:
merged_df = pd.merge(
    merged_df,
    plays[['gameId','playId','pff_manZone']],
    how = "left",
    left_on = ['gameId','playId'],
    right_on = ['gameId','playId']
)

In [152]:
merged_df = merged_df[merged_df['pff_manZone']=="Man"]

In [153]:
week1 = pd.merge(
    merged_playerplay,
    merged_df,
    how = 'right',
    left_on=['gameId','playId','nflId'],
    right_on=['gameId','playId','routeRunnerId']
)

In [154]:
week1 = pd.merge(
    week1,
    players[['nflId','position']],
    how = 'left',
    left_on=['routeRunnerId'],
    right_on=['nflId']
)

In [155]:
startingpos = df[['gameId','playId','nflId','event','x','y']][(df['event']=="ball_snap")]

In [156]:
week1 = pd.merge(
    week1,
    startingpos[['gameId','playId','nflId','x','y']],
    how='left',
    left_on=['gameId','playId','routeRunnerId'],
    right_on=['gameId','playId','nflId']
)

In [157]:
week1motion = pd.read_csv('week1motion.csv')

In [158]:
cluster_means = week1motion.groupby('cluster').agg(
    first_mean=('first', 'mean'),
    last_mean=('last', 'mean'),
    direction_change_mean=('direction_change', 'mean'),
    move_at_snap_mean=('inMotionAtBallSnap','mean'),
    count=('first', 'size')
)

In [159]:
week1man = pd.merge(
    week1[['gameId', 'playId', 'possessionTeam', 'defensiveTeam', 'quarter',
       'down', 'yardsToGo', 'offenseFormation', 'receiverAlignment',
       'passResult', 'playAction', 'timeToThrow', 'pff_passCoverage',
       'pff_manZone_x', 'inMotionAtBallSnap', 'motionSinceLineset',
       'routeRan_x', 'hadPassReception', 'receivingYards', 'routeRunnerId',
       'defenderId', 'position', 'x','y']],
    week1motion[['gameId','playId','nflId','cluster']],
    how = "left",
    left_on = ['gameId','playId','routeRunnerId'],
    right_on = ['gameId','playId','nflId']
)

In [160]:
linesetDefender = pd.read_csv("linesetDefender.csv")

In [161]:
week1man = pd.merge(
    week1man,
    linesetDefender,
    how="left",
    left_on = ['gameId','playId','routeRunnerId'],
    right_on = ['gameId','playId','routeRunnerId']
)

In [162]:
weekOneVoronoi = pd.read_csv('voronoiWeekOne.csv')

In [203]:
weekone = pd.merge(
    week1man,
    weekOneVoronoi,
    how = 'right',
    left_on = ['gameId','playId','routeRunnerId'],
    right_on = ['gameId','playId','routeRunnerId']
)

In [204]:
weekone.drop(columns=['passResult', 'quarter','possessionTeam','defensiveTeam','hadPassReception','receivingYards','nflId','Unnamed: 0','timeToThrow_y','nflId_x', 'position_y','nflId_y','routeRan_x'], inplace=True)

In [205]:
weekone = pd.merge(
    weekone,
    df[['gameId','playId','frameId','nflId','s','a']],
    how = "left",
    left_on = ['gameId','playId','frameId','routeRunnerId'],
    right_on = ['gameId','playId','frameId','nflId']
)

In [207]:
weekone.drop(columns=['gameId','playId','frameId'],inplace=True)

In [209]:
weekone = pd.merge(
    weekone,
    players[['nflId','position']],
    how="left",
    left_on=['defenderId'],
    right_on = ['nflId']
)

In [211]:
weekone.rename(columns={
    'position_x': 'routeRunnerPosition',
    'position': 'atSnapDefenderPosition',
    'timeSinceSnap': 'timeOfMaxSeparation',
    'cluster': 'motionCluster',
    'timeToThrow_x': 'timeToThrow',
    'presnapDefenderId': 'preMotionDefenderId',
    'presnapDefenderPosition': 'preMotionDefenderPosition',
    'pff_manZone_x': 'pff_manZone',
    'defenderId': 'atSnapDefenderId'
}, inplace=True)

In [213]:
weekone.drop(columns=['nflId_x','nflId_y'],inplace=True)

In [215]:
column_order = [
    'routeRunnerId',
    'routeRunnerPosition',
    'x',
    'y',
    'motionCluster',
    's',
    'a',
    'atSnapDefenderId',  # Move nflId_defender to 4th
    'atSnapDefenderPosition',
    'preMotionDefenderId', 
    'preMotionDefenderPosition',
    'down', 
    'yardsToGo', 
    'offenseFormation', 
    'receiverAlignment',
    'playAction', 
    'timeToThrow', 
    'pff_passCoverage', 
    'pff_manZone', 
    'inMotionAtBallSnap', 
    'motionSinceLineset', 
    'routeRan', 
    'absoluteYardlineNumber',
    'timeOfMaxSeparation',
    'maxValueOfSeparation'
]

In [216]:
weekone = weekone[column_order]

In [221]:
weekone.drop(columns=['routeRunnerId','atSnapDefenderId','preMotionDefenderId'],inplace=True)

In [224]:
weekone.to_csv("weekone.csv")